In [ ]:
import datetime
import pandas as pd

In [ ]:
from kiteconnect import KiteConnect

In [ ]:
api_key = "kvvzk6jq4zn4o4vc"
kite = KiteConnect(api_key=api_key)

In [ ]:
auth_url = kite.login_url()
print("Open this URL in your browser to authorize the app:")
print(auth_url)

In [ ]:
access_Token = "K9c2In2fSuBo433ffAVX0hpsEwuYwV53"

In [ ]:
kite = KiteConnect(api_key="kvvzk6jq4zn4o4vc")
data = kite.generate_session(access_Token, api_secret="nzcqxwu05jmo2k71cmpgnbf5trz6gypn")
access_token = data["access_token"]

kite.set_access_token(access_token)

In [ ]:
profile = kite.profile()

# Step 3: Print profile information
print("User ID:", profile["user_id"])
print("User Name:", profile["user_name"])
print("Email:", profile["email"])
print("User type:", profile["user_type"])
print("Products:", profile["products"])
print("Broker info:", profile["broker"])

In [ ]:
# # Get all members of the kite object


# import inspect

# # Get all members of the kite object
# all_members = inspect.getmembers(kite, predicate=inspect.ismethod)

# # Print in a readable format
# for name, func in all_members:
#     print(name)



In [ ]:
instruments = kite.instruments()

In [ ]:
def bull_put_spread(kite, instruments, lower_premium=40, upper_premium=60):
    print("Printing Bull put spread details")
    import datetime

    nifty_fut_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'FUT'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
    ]

    if not nifty_fut_list:
        print("No NIFTY FUT instruments found.")
        return None, None

    # Pick nearest FUT (first sorted by expiry)
    nifty_fut_list = sorted(nifty_fut_list, key=lambda x: x['expiry'])
    selected_future = nifty_fut_list[0]

    # Fetch LTP
    fut_key = f"{selected_future['exchange']}:{selected_future['tradingsymbol']}"
    future_ltp_data = kite.ltp([fut_key])

    future_price = future_ltp_data[fut_key]['last_price']

    future_symbol = selected_future['tradingsymbol']
    future_token  = selected_future['instrument_token']
    print("")
    print(f"Selected FUT: {future_symbol} (Token: {future_token})  LTP: {future_price}")
    print("")


    # ----------------------------------- #
    #         PUT HEDGE SELECTION
    # ----------------------------------- #
    today = datetime.date.today()
    min_expiry = today + datetime.timedelta(days=2)

    nifty_pe_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'PE'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
        and inst.get('expiry') > min_expiry
    ]

    if not nifty_pe_list:
        print("No NIFTY PE instruments meeting expiry criteria.")
        return (future_symbol, future_token), None

    # Limit search to reduce API load
    candidate_puts = nifty_pe_list[:200]

    pe_keys = [f"{opt['exchange']}:{opt['tradingsymbol']}" for opt in candidate_puts]
    pe_ltp_data = kite.ltp(pe_keys)

    put_symbol = None
    put_token  = None

    for opt in candidate_puts:
        key = f"{opt['exchange']}:{opt['tradingsymbol']}"
        price = pe_ltp_data[key]['last_price']

        if lower_premium < price < upper_premium:
            put_symbol = opt['tradingsymbol']
            put_token  = opt['instrument_token']
            print("")
            print(f"Selected PUT hedge: {put_symbol} (Token: {put_token}) Premium: {price}")
            print("")
            break

    if put_symbol is None:
        print("No PUT hedge found within premium range.")
        return (future_symbol, future_token), None

    return (future_symbol, future_token), (put_symbol, put_token)


In [ ]:
(fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread(kite,instruments,lower_premium=40,upper_premium=60)

print("Future Symbol:", fut_symbol)
print("Future Token :", fut_token)
print("Put Symbol   :", put_symbol)
print("Put Token    :", put_token)


In [ ]:
def bear_call_spread(kite, instruments, lower_premium=40, upper_premium=60):
    print("Printing Bull put spread details")
    import datetime

    # ----------------------------------- #
    #       FUTURE SELECTION
    # ----------------------------------- #
    nifty_fut_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'FUT'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
    ]

    if not nifty_fut_list:
        print("No NIFTY FUT instruments found.")
        return None, None

    # Sort by expiry and pick nearest FUT
    nifty_fut_list = sorted(nifty_fut_list, key=lambda x: x['expiry'])
    selected_future = nifty_fut_list[0]

    # Fetch LTP
    fut_key = f"{selected_future['exchange']}:{selected_future['tradingsymbol']}"
    future_ltp_data = kite.ltp([fut_key])

    future_price  = future_ltp_data[fut_key]['last_price']
    future_symbol = selected_future['tradingsymbol']
    future_token  = selected_future['instrument_token']
    print("")
    print(f"Selected FUT: {future_symbol} (Token: {future_token})  LTP: {future_price}")
    print("")


    # ----------------------------------- #
    #          CALL HEDGE SELECTION
    # ----------------------------------- #
    today = datetime.date.today()
    min_expiry = today + datetime.timedelta(days=2)

    nifty_ce_list = [
        inst for inst in instruments
        if inst.get('instrument_type') == 'CE'
        and inst.get('tradingsymbol', '').startswith("NIFTY")
        and inst.get('name') == 'NIFTY'
        and inst.get('expiry') is not None
        and inst.get('expiry') > min_expiry
    ]

    if not nifty_ce_list:
        print("No NIFTY CE instruments meeting expiry criteria.")
        return (future_symbol, future_token), None

    candidate_calls = nifty_ce_list[:200]

    ce_keys = [f"{opt['exchange']}:{opt['tradingsymbol']}" for opt in candidate_calls]
    ce_ltp_data = kite.ltp(ce_keys)

    call_symbol = None
    call_token  = None

    for opt in candidate_calls:
        key = f"{opt['exchange']}:{opt['tradingsymbol']}"
        price = ce_ltp_data[key]['last_price']

        if lower_premium < price < upper_premium:
            call_symbol = opt['tradingsymbol']
            call_token  = opt['instrument_token']
            print("")
            print(f"Selected CALL hedge: {call_symbol} (Token: {call_token}) Premium: {price}")
            print("")
            break

    if call_symbol is None:
        print("No CALL hedge found within premium range.")
        return (future_symbol, future_token), None

    return (future_symbol, future_token), (call_symbol, call_token)


In [ ]:
(fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread(kite,instruments,lower_premium=40,upper_premium=60)

print("Future Symbol:", fut_symbol)
print("Future Token :", fut_token)
print("Put Symbol   :", call_symbol)
print("Put Token    :", call_token)


In [ ]:

# # Define your orders
# orders = [
#     {
#         "tradingsymbol": "INFY",
#         "exchange": "NSE",
#         "transaction_type": "BUY",
#         "quantity": 1,
#         "order_type": "MARKET",
#         "product": "MIS",
#         "variety": "amo"
#     },
#     {
#         "tradingsymbol": "TCS",
#         "exchange": "NSE",
#         "transaction_type": "SELL",
#         "quantity": 1,
#         "order_type": "MARKET",
#         "product": "MIS",
#         "variety": "amo"
#     }
# ]

# # Place orders
# for order in orders:
#     try:
#         order_id = kite.place_order(
#             variety=order["variety"],
#             exchange=order["exchange"],
#             tradingsymbol=order["tradingsymbol"],
#             transaction_type=order["transaction_type"],
#             quantity=order["quantity"],
#             order_type=order["order_type"],
#             product=order["product"]
#         )
#         print(f"Order placed successfully. Order ID: {order_id}")
#     except Exception as e:
#         print(f"Error placing order for {order['tradingsymbol']}: {e}")

In [ ]:


# # Convert to DataFrame
# df = pd.DataFrame(instruments)

# # Optional: preview first few rows
# print(df.head())

# # Export to CSV
# df.to_csv("kite_instruments.csv", index=False)

# print("Instruments saved to kite_instruments.csv")


In [ ]:
tokens = [fut_token, call_token]

In [ ]:
#!python
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)

# Initialise
kws = KiteTicker(api_key, access_token)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    logging.debug("Ticks: {}".format(ticks))

def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    ws.subscribe(tokens)

    # Set RELIANCE to tick in `full` mode.
    ws.set_mode(ws.MODE_FULL, [fut_token])

def on_close(ws, code, reason):
    # On connection close stop the event loop.
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
kws.connect()